In [5]:
import socket
import time 
import numpy as np
import pickle
from pynq import PL
from pynq import allocate
from pynq import Overlay

In [9]:
PL.reset()
ol = Overlay('./design_1.bit')
dma = ol.axi_dma_0

In [14]:
n = 8192
HOST = "192.168.0.100"
PORT = 65432

out_buffer = allocate(n, dtype=np.int32)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    sent_len = False
    loop_start = time.time()
    for i in range(100):
        dma.recvchannel.transfer(out_buffer)
        dma.recvchannel.wait()
        if not sent_len:
            s.sendall(len(pickle.dumps(np.array(out_buffer))).to_bytes(4, 'little'))
            sent_len = True
        pickled = pickle.dumps(np.array(out_buffer))
        for i in range(0, len(pickled), 1024):
            s.sendall(pickled[i:i+1024])
    s.shutdown(socket.SHUT_WR)
    data = s.recv(1024)    

    print("Loop runtime", time.time() - loop_start)    

out_buffer.close()

Loop runtime 9.695027112960815
